# RUL Prediction Video

In [1]:
import numpy as np
import torch
from pickle import load,dump
from datetime import date
import os

# custom modules
from method import compute_eval_matrices
from watercan import GroupWaterCan
from visualization import make_rul_video 
from helpers import get_from

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Get the data

In [3]:
dataset_name='N-CMAPSS' #'uav_data'#
threshold_name=''#'-0.1' #  '+0.1' #

# Load test data

In [4]:
with open(f'./data/{dataset_name}/performs_info.pkl', 'rb') as file:
    performs_info=load(file)

with open(f'./data/{dataset_name}/time_info.pkl', 'rb') as file:
    time_info=load(file)
    
with open(f'./data/{dataset_name}/times_test{threshold_name}.pkl', 'rb') as file:
    t_observ = load(file)

with open(f'./data/{dataset_name}/times_test{threshold_name}_threshold.pkl', 'rb') as file:
    times_thresholds = load(file)

with open(f'./data/{dataset_name}/RUL_test{threshold_name}.pkl', 'rb') as file:
    rul= load(file)

with open(f'./data/{dataset_name}/EOLcause_test{threshold_name}.pkl', 'rb') as file:
    EOLcause= load(file)

performs = {}
thresholds = {}
for name in performs_info.keys():
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}.pkl', 'rb') as file:
        performs[name] = load(file)
    with open(f'./data/{dataset_name}/{name}_test{threshold_name}_threshold.pkl', 'rb') as file:
        thresholds[name]= load(file)

EOLcause

['HPT_eff', 'HPT_eff', 'HPT_eff', 'HPT_eff']

## Choose Library

In [5]:
library_address=f'./Library/MTL_2024-08-14_{dataset_name}/'

## Load Group Distribution

In [6]:
group_distributions={}
for perform_name in performs_info.keys():
    perform_info=performs_info[perform_name]

    perform_address=library_address+f'{perform_name}/'

    configs=torch.load(perform_address+'config.pth')
    model_state=torch.load(perform_address+'model_state.pth')
    model=GroupWaterCan(configs)
    model.load_state_dict(model_state)
    model.to(device)
    group_distributions[perform_name]=model
t_observ_torch=[torch.tensor(time,device=device,dtype=torch.float32).unsqueeze(dim=1) for time in t_observ]

RuntimeError: Error(s) in loading state_dict for GroupWaterCan:
	Missing key(s) in state_dict: "watercans.0.latentf.no_linear.0.0.weight", "watercans.0.latentf.no_linear.0.0.bias", "watercans.0.latentf.no_linear.1.0.weight", "watercans.0.latentf.no_linear.1.0.bias", "watercans.0.latentf.linear.0.weight", "watercans.0.latentf.linear.0.bias", "watercans.0.latentf.linear.1.weight", "watercans.0.latentf.linear.1.bias", "watercans.0.latentf_out.no_linear.0.0.weight", "watercans.0.latentf_out.no_linear.0.0.bias", "watercans.0.latentf_out.no_linear.1.0.weight", "watercans.0.latentf_out.no_linear.1.0.bias", "watercans.0.latentf_out.linear.0.weight", "watercans.0.latentf_out.linear.0.bias", "watercans.0.latentf_out.linear.1.weight", "watercans.0.latentf_out.linear.1.bias", "watercans.1.latentf.no_linear.0.0.weight", "watercans.1.latentf.no_linear.0.0.bias", "watercans.1.latentf.no_linear.1.0.weight", "watercans.1.latentf.no_linear.1.0.bias", "watercans.1.latentf.linear.0.weight", "watercans.1.l

# Compute and prepare A,b matrices and time

In [ ]:
n_test_unit=len(t_observ)
As,Ss=compute_eval_matrices(group_distributions,t_observ_torch)
test_unit_list=range(n_test_unit)
bs = performs

## Choose Experiment parameters

In [ ]:
# Performance-independent parameters
n_train_unit =20 # 6
n_paths = 20    # 10 #
ood_coef=1.
conserv_pred=0.5


# Performance-dependent parameters
gamma={key:perform['default_gamma'] for key,perform in performs_info.items()}
max_life =time_info['max_life']


# Visualization
accel=15*3600
time_unit=time_info['time_unit']
long_life=max([len(time) for time in t_observ])
t = np.linspace(0, int(long_life+5), int((long_life+5)*2.5))

In [ ]:
config={'n_train_unit':n_train_unit,
        'n_paths':n_paths,
        'ood_coef':ood_coef,
        'conserv_pred':conserv_pred,
        'gamma':gamma,
        'max_life':max_life}

In [ ]:
# y_lim_={'HI': (0,1)}
# loc_={'HI': 'lower left'}
# monot_={'HI':'-'}

# time_unit='cycle'
# causes_text = ['HPT_eff']

#### Experiment name

In [ ]:
today = date.today()
threshold_text=0 if not threshold_name else threshold_name

In [ ]:
exp=f'threshold{threshold_text}_{today}/'

exp='try'

if exp == 'try':
    n_train_unit =5 #5#
    n_paths =5 #5# 

#### Save experiment configuration

In [ ]:
exp_address=library_address+f'_Experiments/{exp}/'
os.makedirs(exp_address, exist_ok=True)
with open(exp_address+'config.pth', "wb") as f:
    dump(config, f)

### Make Predictions

In [ ]:
## Setup Method
y_lim=get_from(performs_info,'y_lim')
loc=get_from(performs_info,'loc')
monot=get_from(performs_info,'monot')
EOL_cause_text =get_from(performs_info,'EOL_cause') 

subgroup_distributions={name:group_dist.get_sub_watercan(range(n_train_unit)) for name,group_dist in group_distributions.items()}

## Results initialization
trues = []
pred_names='acc','csv','pred','unc'
RMSEs={key: [] for key in pred_names}
predss={key: [] for key in pred_names}
causess={key: [] for key in pred_names}

n_unit=1 if exp=='try' else n_test_unit
for unit in range(n_unit): 
        A={}
        S={}
        b={}
        threshold={}
        for name in performs_info.keys():
                A[name]= As[name][unit][:,:n_train_unit]
                S[name]= Ss[name][unit][:,:n_train_unit]
                b[name]= bs[name][unit]
                thres=np.interp(t, times_thresholds[unit], thresholds[name][unit])
                threshold[name]=torch.tensor(thres,dtype=torch.float32,device=device)

        preds,causes=make_rul_video(unit,t,t_observ[unit],rul[unit],EOLcause[unit],
                                monot, threshold, subgroup_distributions, A, b, S, gamma, y_lim, loc,
                                perform_names=performs_info.keys(),causes_text=EOL_cause_text,time_unit=time_unit,
                                n_train_unit=n_train_unit, n_paths=n_paths, ood_coef=ood_coef,max_life = max_life,
                                accel=accel//3600, save=exp_address,conserv=conserv_pred)
        y_true=rul[unit]

        for name in pred_names:
                RMSE=np.sqrt(np.mean((preds[name]-y_true)**2))
                print(f"RMSE_{name}: {RMSE}")
                RMSEs[name].append(RMSE)
                predss[name].append(preds[name])
                causess[name].append(causes[name])
        trues.append(y_true)

torch.save(predss,f'{exp_address}'+'preds.pt')
torch.save(causess,f'{exp_address}'+'causes.pt')
torch.save(trues,f'{exp_address}'+'trues.pt')
print('----------------------------------------------------------------')
for name in pred_names:
        print(f"RMSE_{name} mean: {np.mean(RMSEs[name],axis=0)}")
        print(f"RMSE_{name} std: {np.std(RMSEs[name],axis=0)}")
print('----------------------------------------------------------------')

In [ ]:
predss=torch.load(f'{exp_address}/preds.pt')
trues=torch.load(f'{exp_address}/trues.pt')

In [ ]:
for name in pred_names:
        print(f'------------------------------- {name} RMSE -------------------------------')
        RMSEs=[]
        for unit in range(len(trues)):
                RMSE=np.sqrt(np.mean((predss[name][unit]-trues[unit])**2))
                print(f"unit {unit}: {RMSE}")
                RMSEs.append(RMSE)
        print('----------------------------------')
        print(f"RMSE_{name} mean: {np.mean(RMSEs,axis=0)}")
        print(f"RMSE_{name} std: {np.std(RMSEs,axis=0)}")
        print('----------------------------------')

------------------------------- acc RMSE -------------------------------
unit 0: 10.2939106256353
----------------------------------
RMSE_acc mean: 10.2939106256353
RMSE_acc std: 0.0
----------------------------------
------------------------------- csv RMSE -------------------------------
unit 0: 3.1047353156373045
----------------------------------
RMSE_csv mean: 3.1047353156373045
RMSE_csv std: 0.0
----------------------------------
------------------------------- pred RMSE -------------------------------
unit 0: 3.699746887608005
----------------------------------
RMSE_pred mean: 3.699746887608005
RMSE_pred std: 0.0
----------------------------------
------------------------------- unc RMSE -------------------------------
unit 0: 35.30232015421118
----------------------------------
RMSE_unc mean: 35.30232015421118
RMSE_unc std: 0.0
----------------------------------


In [ ]:
exp_address

'./Library/MTL_2024-08-17_uav_data/_Experiments/try/'